In [1]:
path = 'C:/Users/ISHMO_CT/Downloads/my-projects/simple_lin_clas_env/data/um-game-playing-strength-of-mcts-variants/train.csv'

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv(path)

In [4]:
df.head()

,Id,GameRulesetName,agent1,agent2,Properties,Format,Time,Discrete,Realtime,Turns,...,DoLudeme,Trigger,PlayoutsPerSecond,MovesPerSecond,EnglishRules,LudRules,num_wins_agent1,num_draws_agent1,num_losses_agent1,utility_agent1
0,0,00Y,MCTS-ProgressiveHistory-0.1-MAST-false,MCTS-ProgressiveHistory-0.6-Random200-false,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",4,0,11,-0.466667
1,1,00Y,MCTS-ProgressiveHistory-0.1-MAST-false,MCTS-UCB1GRAVE-0.6-NST-true,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",5,0,10,-0.333333
2,2,00Y,MCTS-ProgressiveHistory-0.1-MAST-true,MCTS-UCB1-0.1-NST-false,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",7,0,8,-0.066667
3,3,00Y,MCTS-ProgressiveHistory-0.1-MAST-true,MCTS-UCB1-0.6-NST-false,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",5,0,10,-0.333333
4,4,00Y,MCTS-ProgressiveHistory-0.1-MAST-true,MCTS-UCB1GRAVE-1.41421356237-NST-false,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",5,0,10,-0.333333


In [6]:
# do exploratory data analysis 
print(df.isnull().sum(), df.shape, df.describe(), df.columns, df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233234 entries, 0 to 233233
Columns: 814 entries, Id to utility_agent1
dtypes: float64(201), int64(608), object(5)
memory usage: 1.4+ GB
Id                   0
GameRulesetName      0
agent1               0
agent2               0
Properties           0
                    ..
LudRules             0
num_wins_agent1      0
num_draws_agent1     0
num_losses_agent1    0
utility_agent1       0
Length: 814, dtype: int64 (233234, 814)                   Id  Properties    Format      Time  Discrete  Realtime  \
count  233234.000000    233234.0  233234.0  233234.0  233234.0  233234.0   
mean   116616.500000         1.0       1.0       1.0       1.0       0.0   
std     67329.000679         0.0       0.0       0.0       0.0       0.0   
min         0.000000         1.0       1.0       1.0       1.0       0.0   
25%     58308.250000         1.0       1.0       1.0       1.0       0.0   
50%    116616.500000         1.0       1.0       1.0       1.0  